In [10]:
import pandas as pd
import dash
from dash import Dash, html, dash_table, dcc,  Input, Output, callback
import plotly_express as px
import sweetviz as sv
import folium
# from folium.plugins import MarkerCluster
import numpy as np
import random # pour modifier les coordonnées long et lat

import plotly.graph_objects as go
import sys

In [11]:
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

import des modules du folder "scripte_python"

In [12]:
sys.path.append('../scripte_python')
from graph_premiereFig import premiereFig
from graph_premiereFig import mean_counts_by_year
from graph_deuxiemeFig import deuxiemeFig
from graph_selectPays import button_select_country
from graph_selectPays import selectPays
from mapChloropleth import figure_chloropleth

Dataframes

In [13]:
edf = pd.read_csv("../CSV/productions-consolidees-par-pays-du-groupe-edf.csv",delimiter=';')
paysDF = pd.read_csv("../CSV/Country.csv" )
edf_pays=pd.read_csv("../CSV/edf_AND_country.csv")
iso=pd.read_csv("../CSV/all.csv")
add_iso_to_edf_pays=pd.read_csv("../CSV/edf_AND_country_AND_iso.csv")

In [14]:
edf.columns = [col.strip() for col in edf.columns]

initialisation de la page web Dash 

In [15]:
app = dash.Dash(__name__)

Marquage dynamique pour que les boutons puissent impacter en temps reel les graphs


In [16]:
app.layout = html.Div([
    html.H1("Étude du marché de l'énergie d'EDF au niveau mondial", style={'textAlign': 'center', 'color': '#7FDBFF'}),

    html.Div([
        html.P("Data set appartenant à EDF"),
        dcc.Markdown("[Source Internet](https://opendata.edf.fr/explore/dataset/productions-consolidees-par-pays-du-groupe-edf/information/?disjunctive.perimetre_spatial&disjunctive.filiere&sort=-tri&dataChart=eyJxdWVyaWVzIjpbeyJjaGFydHMiOlt7InR5cGUiOiJjb2x1bW4iLCJmdW5jIjoiU1VNIiwieUF4aXMiOiJwcm9kdWN0aW9uIiwic2NpZW50aWZpY0Rpc3BsYXkiOnRydWUsImNvbG9yIjoiIzAwMUE3MCJ9XSwieEF4aXMiOiJhbm5lZSIsIm1heHBvaW50cyI6bnVsbCwidGltZXNjYWxlIjoieWVhciIsInNvcnQiOiIiLCJzZXJpZXNCcmVha2Rvd25UaW1lc2NhbGUiOiIiLCJjb25maWciOnsiZGF0YXNldCI6InByb2R1Y3Rpb25zLWNvbnNvbGlkZWVzLXBhci1wYXlzLWR1LWdyb3VwZS1lZGYiLCJvcHRpb25zIjp7ImRpc2p1bmN0aXZlLnBlcmltZXRyZV9zcGF0aWFsIjp0cnVlLCJkaXNqdW5jdGl2ZS5maWxpZXJlIjp0cnVlLCJzb3J0IjoiLXRyaSJ9fX1dLCJkaXNwbGF5TGVnZW5kIjp0cnVlLCJhbGlnbk1vbnRoIjp0cnVlLCJ0aW1lc2NhbGUiOiIifQ%3D%3D)")
    ]),

    html.Div([
        html.H2("Data frame de EDF",style={'textAlign': 'center'}),
        dash_table.DataTable(data=edf.to_dict('records'),
                             page_size=10,
                             style_data_conditional=[
                                {
                                    'backgroundColor': 'black',
                                    'color': 'white'   
                                }
                                ],
                            )
    ]),
       html.Div([
        html.H2("Data frame des pays",style={'textAlign': 'center'}),
        dash_table.DataTable(data=paysDF.to_dict('records'),
                             page_size=10,
                             style_data_conditional=[
                                {
                                    'backgroundColor': 'black',
                                    'color': 'white'   
                                }
                                ],
                            )
    ]),

    html.Div([
        html.H2("Data frame EDF + pays",style={'textAlign': 'center'}),
        dash_table.DataTable(data=edf_pays.to_dict('records'), page_size=10,
                             style_data_conditional=[
                                {
                                    'backgroundColor': 'black',
                                    'color': 'white'   
                                }
                                ],
                            )
    ]),
    
    html.Div([
        html.H2("Data frame de EDFzzzzzzzzzzzzzzzzzzzz",style={'textAlign': 'center'}),
        dcc.Graph(figure=premiereFig())
    ]),
    
    html.Div([
        html.H2("Data frame de EDF",style={'textAlign': 'center'}),
        dash_table.DataTable(data=mean_counts_by_year().to_dict('records'), page_size=10)
    ]),
    
    html.Div([
        html.H2("2ieme graph edf",style={'textAlign': 'center'}),
        dcc.Graph(figure=deuxiemeFig())
    ]),
    
    html.Div([
    html.H2("Détail de la production d'énergie d'EDF par pays",style={'textAlign': 'center', 'backgroundColor': 'black',
                                    'color': 'white' }),
    dcc.Dropdown(
        id='country-dropdown',
        options=[{'label': country, 'value': country}
        for country in edf['Périmètre spatial'].unique()],
        value=edf['Périmètre spatial'].unique()[0],  # Default value
        multi=False,

        style={'width': '50%', 'color': 'blue', 'backgroundColor': 'grey' },
        placeholder='Choisir un pays',
                                            #je ne peux paas le mettre dans un def dans graph_selectedPays.py
    ),  
    dcc.Graph( id='diagramme')   
    ]), 
                     
    html.Div([
        html.H1("Energy Production Visualization"),
        dcc.Graph(figure=figure_chloropleth())
    ])   
])






In [17]:
@app.callback(
    Output('diagramme', 'figure'),
    Input('country-dropdown', 'value')
)
def update_diagram(selected_country):
    filtered_df = edf[edf['Périmètre spatial'] == selected_country]
    filtered_df['Année'] = filtered_df['Année'].astype('category')
    fig = px.scatter(
        filtered_df,
        x='Année',
        y='Production',
        render_mode='svg',
        # style = {'backgroundColor': '#FFD700'},
        labels={'Année': 'Année', 'Production': 'Production'},
        template='plotly_white',
        color_discrete_sequence=['blue']
    )
    # Forcer Plotly Express à traiter l'axe x comme catégorie
    fig.update_xaxes(type='category')
    # fig.update_layout( plot_bgcolor='linear-gradient(180deg, #000000, #404040)')
    return fig


C:\Users\jonat\AppData\Local\Temp\ipykernel_20916\1225712296.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

